In [1]:
import pandas as pd
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds

In [2]:
# sample 데이터를 이용한 영화 추천 시스템

# 데이터 읽어오기
df = pd.read_csv('111.csv', encoding = 'cp949')

print(df.shape)
df.head(3)

(10128, 4)


,num,title,score,NAME2
0,16675534,미운 오리 새끼,10,pig*****
1,16675535,부산행,8,edw*****
2,16675537,붉은 돼지,10,fbt*****


In [3]:
df['NAME2'] = df['NAME2'].apply(lambda x : x[:3])
df = df.drop_duplicates(['title', 'NAME2'], keep='first')[['title', 'score','NAME2']]
df.columns = ["movieId","rating","userId"]
df = df[['userId', 'movieId', 'rating']]
print(df.shape)
df.head(3)

(9201, 3)


,userId,movieId,rating
0,pig,미운 오리 새끼,10
1,edw,부산행,8
2,fbt,붉은 돼지,10


In [29]:
df[df['movieId'] == '나이브스 아웃']['rating'].mean()

9.142857142857142

In [4]:
df['userId'].unique()

array(['pig', 'edw', 'fbt', ..., 'knh', 'vam', 'hkj'], dtype=object)

In [31]:
def find_top_lst_movie(df, userid, movie_len):

    # 전체 영화 목록
    total_movie_list = set(df.movieId)
    # 유저가 본 영화 목록
    movie_list = df[df['userId'] == userid]['movieId']
    # 유저가 보지 않은 영화 목록
    find_movie_list = total_movie_list - set(movie_list)

    # 데이터 로드
    reader = Reader(rating_scale=(0.0, 10.0))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    
    # 추천 행렬 분해 알고리즘으로 SVD객체를 생성하고 학습수행
    algo = SVD()
    train = data.build_full_trainset()
    algo.fit(train)
    
    pred = []
    for iid in total_movie_list:
        pred.append(algo.predict(userid, iid, verbose=False))

    result = pd.DataFrame([[i.uid, i.iid, i.est] for i in pred], columns=df.columns[:3])

    result.sort_values(by = "rating",ascending = False, inplace = True)

    top_lst = []

    print("{}님에게 {}개의 영화 추천 목록".format(userid, movie_len))
    print("--------------------------------")

    for i in result.values:
        if len(top_lst) == movie_len:
            break
        if i[1] in find_movie_list:
            print("{}".format(i[1]))
            top_lst.append(i)

In [32]:
find_top_lst_movie(df, 'edw', 5)

edw님에게 5개의 영화 추천 목록
--------------------------------
8월의 크리스마스
나는 전설이다
원더
타이타닉
인터스텔라


In [34]:
def find_top_lst_movie__(df, userid, movie_len):

    # 전체 영화 목록
    total_movie_list = set(df.movieId)
    # 유저가 본 영화 목록
    movie_list = df[df['userId'] == userid]['movieId']
    # 유저가 보지 않은 영화 목록
    find_movie_list = total_movie_list - set(movie_list)

    # 데이터 로드
    reader = Reader(rating_scale=(0.0, 10.0))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    
    # 추천 행렬 분해 알고리즘으로 SVD객체를 생성하고 학습수행
    algo = SVD()
    train = data.build_full_trainset()
    algo.fit(train)
    
    pred = []
    for iid in total_movie_list:
        pred.append(algo.predict(userid, iid, verbose=False))

    result = pd.DataFrame([[i.uid, i.iid, i.est] for i in pred], columns=df.columns[:3])

    result.sort_values(by = "rating",ascending = False, inplace = True)
    
    return result

    top_lst = []

    print("{}님에게 {}개의 영화 추천 목록".format(userid, movie_len))
    print("--------------------------------")

    for i in result.values:
        if len(top_lst) == movie_len:
            break
        if i[1] in find_movie_list:
            print("{}".format(i[1]))
            top_lst.append(i)

In [39]:
result = find_top_lst_movie__(df, 'edw', 5)

In [67]:
movie = ['클로젯','버즈 오브 프레이(할리 퀸의 황홀한 해방)','남산의 부장들',
         '히트맨', '극장판 미니특공대: 공룡왕 디노','인셉션', '타오르는 여인의 초상',
         '스파이 지니어스', '페인 앤 글로리', '이 멋진 세계에 축복을! 붉은 전설','조조 래빗', 
         '미스터 주: 사라진 VIP', '사마에게','겨울왕국 2']

In [68]:
def func(data):
    if data in movie:
        return True
    return False

result[result['movieId'].apply(func)]

,userId,movieId,rating
696,edw,타오르는 여인의 초상,9.963136
721,edw,인셉션,9.704682
1231,edw,극장판 미니특공대: 공룡왕 디노,9.080742
137,edw,스파이 지니어스,8.968249
312,edw,조조 래빗,8.955645
1447,edw,이 멋진 세계에 축복을! 붉은 전설,8.646830
445,edw,남산의 부장들,8.535491
113,edw,겨울왕국 2,8.320371
1179,edw,사마에게,7.442381
384,edw,클로젯,7.211977


In [7]:
def find_top_lst_movie_(df, userid, movie_len):

    # 전체 영화 목록
    total_movie_list = set(df.movieId)
    # 유저가 본 영화 목록
    movie_list = df[df['userId'] == userid]['movieId']
    # 유저가 보지 않은 영화 목록
    find_movie_list = total_movie_list - set(movie_list)

    # 데이터 로드
    reader = Reader(rating_scale=(0.0, 10.0))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
    
    # 추천 행렬 분해 알고리즘으로 SVD객체를 생성하고 학습수행
    algo = SVD()
    train = data.build_full_trainset()
    algo.fit(train)
    
    pred = []
    for iid in total_movie_list:
        pred.append(algo.predict(userid, iid, verbose=False))

    result = pd.DataFrame([[i.uid, i.iid, i.est] for i in pred], columns=df.columns[:3])

    result.sort_values(by = "rating",ascending = False, inplace = True)

    top_lst = []

    print("{}님에게 {}개의 영화 추천 목록".format(userid, movie_len))

    for i in result.values:
        if len(top_lst) == 10:
            break
        if i[1] in find_movie_list:
            #print(i[1], i[2])
            #print("{}".format(i[1]))
            top_lst.append(i[1])
            
    print(top_lst)
    
    return top_lst

In [8]:
Movie_Recommendation_by_User = {}

Movie_Recommendation_by_User['pig'] = find_top_lst_movie_(df, 'pig', 10)

pig님에게 10개의 영화 추천 목록
['인셉션', '인터스텔라', '타이타닉', '8월의 크리스마스', '컨테이젼', '싱글라이더', '조조 래빗', '코코', '포드 V 페라리', '그것만이 내 세상']


In [9]:
Movie_Recommendation_by_User = {}

cnt = 0

for i in df['userId'].unique():
    if cnt == 100:
        break
    Movie_Recommendation_by_User[i] = find_top_lst_movie_(df, i, 10)
    cnt += 1

pig님에게 10개의 영화 추천 목록
['나는 전설이다', '타이타닉', '컨테이젼', '코코', '포드 V 페라리', '극장판 미니특공대: 공룡왕 디노', '8월의 크리스마스', '인셉션', '인터스텔라', '조커']
edw님에게 10개의 영화 추천 목록
['8월의 크리스마스', '타이타닉', '원더', '컨테이젼', '코코', '스파이 지니어스', '인셉션', '리틀 포레스트', '극장판 미니특공대: 공룡왕 디노', '싱글라이더']
fbt님에게 10개의 영화 추천 목록
['원더', '8월의 크리스마스', '타이타닉', '인셉션', '스파이 지니어스', '인터스텔라', '조조 래빗', '극장판 미니특공대: 공룡왕 디노', '그것만이 내 세상', '컨테이젼']
p27님에게 10개의 영화 추천 목록
['원더', '타이타닉', '타오르는 여인의 초상', '컨테이젼', '인셉션', '그것만이 내 세상', '8월의 크리스마스', '극장판 미니특공대: 공룡왕 디노', '나는 전설이다', '싱글라이더']
wlg님에게 10개의 영화 추천 목록
['8월의 크리스마스', '인터스텔라', '조조 래빗', '원더', '타이타닉', '코코', '타오르는 여인의 초상', '아포칼립토', '그것만이 내 세상', '극장판 미니특공대: 공룡왕 디노']
cur님에게 10개의 영화 추천 목록
['원더', '인셉션', '8월의 크리스마스', '스파이 지니어스', '코코', '나는 전설이다', '타이타닉', '조조 래빗', '극장판 미니특공대: 공룡왕 디노', '미안해요, 리키']
kim님에게 10개의 영화 추천 목록
['타오르는 여인의 초상', '8월의 크리스마스', '미드웨이', '매드맥스: 분노의 도로', '월터의 상상은 현실이 된다', '결혼 이야기', '지금 만나러 갑니다', '노팅 힐', '그것만이 내 세상', '알라딘']
koo님에게 10개의 영화 추천 목록
['나는 전설이다', '원더', '8월의 크리스마스', '나이브스 아웃', '타이타닉', '극장판 미니특공대: 공룡왕 디노',

tek님에게 10개의 영화 추천 목록
['원더', '타이타닉', '인터스텔라', '극장판 미니특공대: 공룡왕 디노', '8월의 크리스마스', '미드웨이', '조조 래빗', '그것만이 내 세상', '나이브스 아웃', '포드 V 페라리']
hay님에게 10개의 영화 추천 목록
['8월의 크리스마스', '인터스텔라', '원더', '코코', '그것만이 내 세상', '나이브스 아웃', '타이타닉', '포드 V 페라리', '조조 래빗', '극장판 미니특공대: 공룡왕 디노']
jih님에게 10개의 영화 추천 목록
['원더', '인터스텔라', '8월의 크리스마스', '극장판 미니특공대: 공룡왕 디노', '타이타닉', '나이브스 아웃', '인셉션', '그것만이 내 세상', '조조 래빗', '코코']
sma님에게 10개의 영화 추천 목록
['8월의 크리스마스', '타이타닉', '인셉션', '황해', '원더', '인터스텔라', '극장판 미니특공대: 공룡왕 디노', '결혼 이야기', '싱글라이더', '그것만이 내 세상']
mid님에게 10개의 영화 추천 목록
['컨테이젼', '원더', '8월의 크리스마스', '조조 래빗', '인터스텔라', '타이타닉', '나는 전설이다', '인셉션', '싱글라이더', '코코']
94w님에게 10개의 영화 추천 목록
['원더', '8월의 크리스마스', '인셉션', '조조 래빗', '나는 전설이다', '타이타닉', '그것만이 내 세상', '코코', '인터스텔라', '미안해요, 리키']
123님에게 10개의 영화 추천 목록
['원더', '8월의 크리스마스', '컨테이젼', '인셉션', '타이타닉', '인터스텔라', '극장판 미니특공대: 공룡왕 디노', '스파이 지니어스', '그것만이 내 세상', '나는 전설이다']
vud님에게 10개의 영화 추천 목록
['원더', '8월의 크리스마스', '인터스텔라', '기생충', '타이타닉', '조조 래빗', '극장판 미니특공대: 공룡왕 디노', '그것만이 내 세상', '인셉션', '코코']
tae님에게 10개의 영화

In [10]:
Movie_Recommendation_by_User

{'pig': ['나는 전설이다',
  '타이타닉',
  '컨테이젼',
  '코코',
  '포드 V 페라리',
  '극장판 미니특공대: 공룡왕 디노',
  '8월의 크리스마스',
  '인셉션',
  '인터스텔라',
  '조커'],
 'edw': ['8월의 크리스마스',
  '타이타닉',
  '원더',
  '컨테이젼',
  '코코',
  '스파이 지니어스',
  '인셉션',
  '리틀 포레스트',
  '극장판 미니특공대: 공룡왕 디노',
  '싱글라이더'],
 'fbt': ['원더',
  '8월의 크리스마스',
  '타이타닉',
  '인셉션',
  '스파이 지니어스',
  '인터스텔라',
  '조조 래빗',
  '극장판 미니특공대: 공룡왕 디노',
  '그것만이 내 세상',
  '컨테이젼'],
 'p27': ['원더',
  '타이타닉',
  '타오르는 여인의 초상',
  '컨테이젼',
  '인셉션',
  '그것만이 내 세상',
  '8월의 크리스마스',
  '극장판 미니특공대: 공룡왕 디노',
  '나는 전설이다',
  '싱글라이더'],
 'wlg': ['8월의 크리스마스',
  '인터스텔라',
  '조조 래빗',
  '원더',
  '타이타닉',
  '코코',
  '타오르는 여인의 초상',
  '아포칼립토',
  '그것만이 내 세상',
  '극장판 미니특공대: 공룡왕 디노'],
 'cur': ['원더',
  '인셉션',
  '8월의 크리스마스',
  '스파이 지니어스',
  '코코',
  '나는 전설이다',
  '타이타닉',
  '조조 래빗',
  '극장판 미니특공대: 공룡왕 디노',
  '미안해요, 리키'],
 'kim': ['타오르는 여인의 초상',
  '8월의 크리스마스',
  '미드웨이',
  '매드맥스: 분노의 도로',
  '월터의 상상은 현실이 된다',
  '결혼 이야기',
  '지금 만나러 갑니다',
  '노팅 힐',
  '그것만이 내 세상',
  '알라딘'],
 'koo': ['나는 전설이다',
  '원더',
  '8월의 크리스마스',
  '나이브